In [1]:
#from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D,Input,Activation,Add
from keras.layers import Flatten,Dense,LSTM,Bidirectional,GlobalAveragePooling1D,Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.optimizers import Adam



Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(Conv1D(18, 7, activation='relu', input_shape=(201,1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(18, 7, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 195, 18)           144       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 97, 18)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 91, 18)            2286      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 45, 18)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 810)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               81100     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [2]:
def Conv_BN(input_name,num_filter,kernel_size):
#     x = Conv1D(18, 7)(input_name)
    x = Conv1D(num_filter,kernel_size,padding='same')(input_name)
    x1 = BatchNormalization()(x)
    return Activation('relu')(x1)

def Conv_Res(input_name,num_filter,kernel_size):
    x = Conv_BN(input_name,num_filter,kernel_size)
    x1 = Conv_BN(x,num_filter,kernel_size)
    return Add()([input_name,x1])

def Conv_R_A(input_name,num_filter,kernel_size):
    x = Conv_BN(input_name,num_filter,kernel_size)
    x1 = Conv_BN(x,num_filter,kernel_size)
    return Add()([input_name,x1])

In [3]:
from numpy import random
s = Input((150,28))

layer = GlobalAveragePooling1D()(s)
print(layer.shape)

(None, 28)


In [3]:
input1 = Input(shape=(300,1))
input2 = Input(shape=(550,1))
conv1 = Conv_BN(input1,16,32)

conv_r1 = Conv_Res(conv1,16,32)
conv2  = Conv1D(32,32,strides=2,padding='same')(conv_r1)
conv2_x = BatchNormalization()(conv2)
conv2_y = Activation('relu')(conv2_x)
conv_r2 = Conv_Res(conv2_y,32,32)
conv_r3 = Conv_Res(conv_r2,32,32)

conv3  = Conv1D(64,32,strides=2,padding='same')(conv_r3)
conv3_x = BatchNormalization()(conv3)
conv3_y = Activation('relu')(conv3_x)
conv_r4 = Conv_Res(conv3_y,64,32)

bl = Bidirectional(LSTM(units=20,return_sequences=True))(conv_r4)
gap1 = GlobalAveragePooling1D()(bl)

drop = Dropout(0.5)(gap1)

model1 = Model(input1,drop)
m2 = Sequential()
m2.add(model1)
# m2.add(Flatten())
m2.add(Dense(100, activation='relu'))
m2.add(Dense(5, activation='softmax'))
m2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 40)                507632    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505       
Total params: 512,237
Trainable params: 511,437
Non-trainable params: 800
_________________________________________________________________


In [7]:
model = Sequential()
model.add(Conv1D(18, 7, activation='relu', input_shape=(190,1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(18, 7, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 184, 18)           144       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 92, 18)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 86, 18)            2286      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 43, 18)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 774)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               77500     
_________________________________________________________________
dense_8 (Dense)              (None, 5)                

In [10]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model
import keras.layers

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
v_model = Model(input_img,output)
v_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 256, 256, 64) 256         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 256, 256, 64) 256         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 256, 256, 3)  0           input_3[0][0]                    
____________________________________________________________________________________________